# Assiment 10

#### Get the 15 variables from this raster for all Peru departments polygons. This is the link where shapefiles are located. This is the link of the source raster. The values should be the percentage of district area cover by this specific Morphological Settlement Zone.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install rasterio

In [ ]:
# requirementss
%pip install geopandas matplotlib shapely rasterio numpy pandas sklearn-xarray -q
%pip install git+https://github.com/jgrss/geowombat  -q

In [ ]:
%pip install rasterstats

In [ ]:
!pip install geopandas

In [ ]:
import geopandas as gpd
from rasterstats import zonal_stats
import pandas as pd
import geowombat as gw

import rasterio
import matplotlib.pyplot as plt
from rasterio.merge import merge
from rasterio.plot import show
from rasterio.mask import mask

import geopandas as gpd
from geopandas import GeoSeries
from shapely.geometry import Point, LineString
import numpy as np
import chardet

import pyproj
from shapely.ops import transform

from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.patheffects as pe

In [ ]:
shapefile_path = '/content/drive/MyDrive/TIF/INEI_LIMITE_DEPARTAMENTAL.shp'
department = gpd.read_file(shapefile_path)
print(department.head())

In [ ]:
transformer = pyproj.Transformer.from_crs('epsg:4326', 'esri:54009', always_xy=True)

# Define a function to apply the transformation
def apply_transform(geom):
    return transform(transformer.transform, geom)

# Apply the transformation to the geometries
department['geometry'] = department['geometry'].apply(apply_transform)

In [ ]:
print(department.head())

### Prueba 1

In [ ]:
import geowombat as gw
import geopandas as gpd

polys = gpd.read_file('/content/drive/MyDrive/TIF/INEI_LIMITE_DEPARTAMENTAL.shp')

with gw.open('/content/drive/MyDrive/TIF/GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R10_C11.tif') as src:
    print(src)
    clipped = src.gw.clip_by_polygon(
        polys,
        mask_data=True,
        expand_by=1
    )
    print(clipped)

In [ ]:
# Extraer los valores de píxeles y las coordenadas
raster_values = clipped.values.flatten()
x_coords, y_coords = clipped.xy(np.arange(clipped.width), np.arange(clipped.height))

# Crear un DataFrame de pandas
df = pd.DataFrame({
    'x': x_coords.flatten(),
    'y': y_coords.flatten(),
    'valor_raster': raster_values
})

# Imprimir el DataFrame resultante
print(df)

#### Prueba 2

In [ ]:
from rasterio.windows import from_bounds

# Ensure the shapefile is in the same CRS as the raster
raster_path = '/content/drive/MyDrive/TIF/GHS_BUILT_C_MSZ_E2018_GLOBE_R2023A_54009_10_V1_0_R10_C10.tif'
with rasterio.open(raster_path) as src:

    # Create a mask for the intersection area
    shapes = [feature["geometry"] for _, feature in department.iterrows()]
    out_image, out_transform = mask(src, shapes, crop=True)
    out_meta = src.meta.copy()

# Update the metadata for the new (clipped) raster
out_meta.update({
    "driver": "GTiff",
    "height": out_image.shape[1],
    "width": out_image.shape[2],
    "transform": out_transform
})

# Save the clipped raster to a new file
output_raster_path = '/content/sample_data/raster_1.tif'
with rasterio.open(output_raster_path, 'w', **out_meta) as dst:
    dst.write(out_image)

In [ ]:
import geopandas as gpd
import pandas as pd
import geowombat as gw

# Ruta al archivo raster
raster_path = '/content/sample_data/raster_1.tif'

# Recortar el raster por el polígono de interés utilizando GeoWombat
with gw.open(raster_path) as src:
    clipped = src.gw.clip_by_polygon(
        department,
        mask_data=True
    )

# Extraer los valores de píxeles y las coordenadas
raster_values = clipped.values.flatten()
x_coords, y_coords = clipped.xy(np.arange(clipped.width), np.arange(clipped.height))

# Crear un DataFrame de pandas
df = pd.DataFrame({
    'x': x_coords.flatten(),
    'y': y_coords.flatten(),
    'valor_raster': raster_values
})

# Imprimir el DataFrame resultante
print(df)